In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import pandas as pd
train = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/test.csv', index_col='id')
submission = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/sample_submission.csv', index_col='id')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/porto-seguro-safe-driver-prediction/train.csv'

In [ ]:
# 327p 데이터 합치기 / 타깃 제거
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1)
all_data

In [ ]:
all_features = all_data.columns
all_features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_features = [feature for feature in all_features if 'cat' in feature]

# 생성
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])
encoded_cat_matrix

In [ ]:
# 다음 항목 제거
# ps_ind_14, ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin, ps_car_14
drop_features=['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 
                'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

remaining_features = [feature for feature in all_features
                        if ('cat' not in feature and 
                            'calc' not in feature and 
                            feature not in drop_features)]
remaining_features 

In [ ]:
# 합치기 329
from scipy import sparse
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')
# 데이터 분리
num_train = len(train)
x = all_data_sprs[:num_train]
x_test = all_data_sprs[num_train:]
y = train['target'].values

In [ ]:
# 함수 만들기 331
# 정규화 : 값 범위를 0~1 사이에 표시하도록 만들어줍니다.
import numpy as np

def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    l_mid = np.linespce(1 / n_samples, 1, n_samples)
    
    pred_order = y_true[y_pred.argsort()]
    l_pred = np.cumsum(pred_order) / np.sum(pred_order)
    g_pred = np.sum(l_mid - l_pred)

    true_order = y_true[y_true.argsort()]
    l_true = np.cumsum(true_order) / np.sum(true_order)
    g_true = np.sum(l_mid - l_true)

    return g_pred / g_true
    

def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

In [ ]:
# 335
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)
params = {'objective' : 'binary', 'learning_rate':0.01, 'force_row_wirse':True, 'random_state':0}
oof_val_preds = np.zeros(x.shape[0])
oof_test_preds = np.zeros(x_test.shape[0])

In [ ]:
import lightgbm as lgb

for idx, (train_idx, valid_idx) in enumerate(folds.split(x, y)) :
    print('#'*40, f'폴드 {idx+1} / 폴드 {fold.n_splits}', '#'*40)

In [ ]:
# oof_val_preds[valid_idx]
eval_gini(y, oof_val_preds)

In [ ]:
########### 버전문제로 XG BOOST로 변경합니다. 
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)


import xgboost as xgb
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(
    x, y, test_size=0.2, random_state=0
)
bayes_dtrain = xgb.DMatrix(x_train, y_train)
bayes_dvalid = xgb.DMatrix(x_valid, y_valid)

param_bounds = {'max_depth':(4,8),
                'subsample':(0.6,0.9),
                'colsample_bytree':(0.7,1.0),
                'min_child_weight':(5,7),
                'gamma':(8,11),
                'reg_alpha':(7,9),
                'reg_lambda':(1.1,1.5),
                'scale_pos_weight':(1.4,1.6)
               }
fixed_params = {'objective': 'binary:logistic',
                'learning_rate':0.02,
                'random_state':1991}

In [ ]:
#360
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight, reg_alpha, 
                  gamma, reg_lambda, scale_pos_weight):
    params = {'max_depth':int(round(max_depth)),
                'subsample':subsample,
                'colsample_bytree':colsample_bytree,
                'min_child_weight':min_child_weight,
                'gamma':gamma,
                'reg_alpha':reg_alpha,
                'reg_lambda':reg_lambda,
                'scale_pos_weight':scale_pos_weight}
    params.update(fixed_params)
    

    xgb_model = xgb.train(params=params, dtrain=bayes_dtrain, num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True, feval=gini, early_stopping_rounds=200,
                          #varbose_eval=False
                         )
    best_iter = xgb_model.best_iteration
    preds = xgb_model.predict(bayes_dvalid, iteration_range=(0, best_iter))

    #계산
    gini_score = eval_gini(y_valid, preds)
    gini_score
    return gini_score

In [ ]:
#362
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds,random_state=0)
optimizer.maximize(init_points=3, n_iter=6)

In [ ]:
# 결과 확인 362
max_params = optimizer.max['params']
max_params

In [ ]:
#363~364
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)
oof_val_preds=np.zeros(x.shape[0])
oof_test_preds = np.zeros(x_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(x,y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)

    x_train, y_train = x[train_idx], y[train_idx]
    x_valid, y_valid = x[train_idx], y[train_idx]

    dtrain = xgb.DMatrix(x_train, y_train)
    dvalid = xgb.DMatrix(x_valid, y_valid)
    dtest = xgb.DMatrix(x_test)

    #훈련
    xgb_model = xgb.train(params=max_params, 
                          dtrain = dtrain, 
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100
                         )

    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest, iteration_range=(0, best_iter)) / folds.n_splits

    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, iteration_range=(0, best_iter))
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

eval_gini(y, oof_val_preds)

In [ ]:
# 제출
submission['targer'] = oof_test_preds
submission.to_csv('submission.csv')